<a href="https://colab.research.google.com/github/McTurner45/Attendence-Registration-With-Face-Refognition-and-Execell/blob/main/opelo_final_year_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Dlib

In [ ]:
pip install face_recognition

In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [3]:
path='/content/ImagesAttendence'
images=[]
classNames=[]
myList=os.listdir(path)

In [4]:
for cl in myList:
  currImag=cv2.imread(f'{path}/{cl}')
  images.append(currImag)
  classNames.append(os.path.splitext(cl)[0])

In [5]:
def findEncodings(images):
  encodingList=[]
  for img in images:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    encode= face_recognition.face_encodings(img)[0]
    encodingList.append(encode)

In [6]:
encodeListKnown=findEncodings(images)

In [8]:
def markAttendence(name):
  with open('/content/Attendence.csv') as f:
    myDataList = f.readlines()
    nameList=[]
    for line in myDataList:
      entry=line.split(,)
      nameList.append(entry[0])

    if name not in nameList:
      now= datetime.now()
      dtString= now.strftime('%H:%M:%S')
      f.writelines(f'\n{name},{dtString}')

In [ ]:
cap=cv2.VideoCapture(0)

while True:
  success, img = cap.read()
  imgS= cv2.resize(img,(0,0),None,0.25,0.25)
  imgS=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  facesCurFrame=face_recognition.face_locations(imgS)
  encodeCurFrame= face_recognition.face_encodings(imgS,facesCurFrame)

  for encodeFace,faceLoc in zip(encodeCurFrame,facesCurFrame):
    matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
    faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
    matchIndex= np.argmin(faceDis)

    if matches[matchIndex]:
      name= classNames[matchIndex].upper()

      y1,x2,y2,x1= faceLoc

      y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4

      cv2.ractangle(img,(x1,y1),(x2,y2),(0,255,0),2)
      cv2.ractangle(img,(x1,y2-55),(x2,y2),(0,255,0),cv2.FILLED)
      cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

      markAttendence(name)

  cv2.imshow('Webcam',img)
  cv2.waitKey(1)
  